<a href="https://colab.research.google.com/github/dgo2dance/qwen-tune/blob/main/StableDiffusionServerNgrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion FastAPI Server using https://github.com/shyamsn97/stable-diffusion-server

### Install Weights and send to google drive (optional)

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/CompVis/stable-diffusion-v1-4

# src_path = "/content/stable-diffusion-v1-4"
# dest_path = 'drive/MyDrive/stable-diffusion-pretrained/'

# !cp -r $src_path $dest_path

### Mount Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !ls

drive  sample_data


### Install packages

In [ ]:
!nvidia-smi

Thu Dec  8 04:45:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install sd-server

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 27.6 MB/s 
     |████████████████████████████████| 55 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 970 kB/s 
     |████████████████████████████████| 453 kB 58.0 MB/s 
     |████████████████████████████████| 63 kB 752 kB/s 
     |████████████████████████████████| 5.8 MB 18.9 MB/s 
     |████████████████████████████████| 191 kB 45.6 MB/s 
     |████████████████████████████████| 80 kB 4.6 MB/s 
     |████████████████████████████████| 182 kB 87.2 MB/s 
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 7.6 MB 69.8 MB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=956efb33aa85304640f24e930343451246be4cc9cb05f2c8c9096c2b2b6217ff
  Stored 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


### Server

#### GLOBAL CONSTANTS

In [ ]:
import torch
# STABLE_DIFFUSION_PATH = "drive/MyDrive/stable-diffusion-pretrained/stable-diffusion-v1-4" # if you decided to download the weights directly into google drive
STABLE_DIFFUSION_PATH = "runwayml/stable-diffusion-v1-5"
pipeline_kwargs = {
    "revision":"fp16",
    "torch_dtype":torch.float16
}

#### ngrok auth token can be found after creating a free ngrok account: https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
NGROK_AUTH_TOKEN = "2GWJ6Kw7Sd9RLBXGdfghptrHOje_6qs7PCWeMReNWLYxLUoir" #@param {type:"string"}


#### Create Server

In [ ]:
from sd_server import DiffusersServer

In [ ]:
device = torch.device('cuda')

In [ ]:
server = DiffusersServer.create(
    pretrained_path=STABLE_DIFFUSION_PATH,
    pipeline_kwargs=pipeline_kwargs,
    enable_attention_slicing=True,
    device=device
)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [ ]:
url = server.start(ngrok_auth_token=NGROK_AUTH_TOKEN)

Public URL: https://910f-34-126-175-105.ngrok.io


INFO:     Started server process [72]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_image?prompt=a+photo+of+an+astronaut+riding+a+horse+on+mars HTTP/1.1" 404 Not Found
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_image HTTP/1.1" 404 Not Found
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_image?prompt=a+photo+of+an+astronaut+riding+a+horse+on+mars HTTP/1.1" 404 Not Found
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_prompts HTTP/1.1" 200 OK
PROMPT a photo of an astronaut riding a horse on mars
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_images?prompt=a%20photo%20of%20an%20astronaut%20riding%20a%20horse

  0%|          | 0/51 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


...

  0%|          | 0/51 [00:00<?, ?it/s]

INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "POST /pipeline HTTP/1.1" 200 OK
PROMPT hyperrealistic full length portrait of gorgeous goddess | standing in field full of flowers | detailed gorgeous face! ! | full body! ! | skimpy armor | god rays | intricate | elegant | realistic | hyperrealistic | cinematic | character design | concept art | highly detailed | illustration | digital art | digital painting | depth of field
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_images?prompt=hyperrealistic+full+length+portrait+of+gorgeous+goddess+%7C+standing+in+field+full+of+flowers+%7C+detailed+gorgeous+face%21+%21+%7C+full+body%21+%21+%7C+skimpy+armor+%7C+god+rays+%7C+intricate+%7C+elegant+%7C+realistic+%7C+hyperrealistic+%7C+cinematic+%7C+character+design+%7C+concept+art+%7C+highly+detailed+%7C+illustration+%7C+digital+art+%7C+digital+painting+%7C+depth+of+field HTTP/1.1" 200 OK


  0%|          | 0/51 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "POST /pipeline HTTP/1.1" 200 OK
PROMPT a matte painting of an insanely beautiful female goddess of seduction, sharp focus on eyes, insanely detailed hair, symmetrical, wet luscious lips, few water droplets, intricate details, professionally retouched, elegant, 8k high definition, by artgerm and greg Rutkowski, lighting by albert Bierstadt
INFO:     2601:647:c803:5e90:b335:ac3a:d6a8:bb08:0 - "GET /get_images?prompt=a+matte+painting+of+an+insanely+beautiful+female+goddess+of+seduction%2C+sharp+focus+on+eyes%2C+insanely+detailed+hair%2C+symmetrical%2C+wet+luscious+lips%2C+few+water+droplets%2C+intricate+details%2C+professionally+retouched%2C+elegant%2C+8k+high+definition%2C+by+artgerm+and+greg+Rutkowski%2C+lighting+by+albert+Bierstadt HTTP/1.1" 200 OK
PROMPT a matte painting of an insanely beautiful female goddess of seduction, sharp focus on eyes, insanely detailed hair, symmetrical, wet luscious lips, few water droplets, intricate de

Now you can connect to the server using the client on another host:

In [ ]:
prompt_image_dict['a photo of an astronaut riding a horse on mars']

[<PIL.Image.Image image mode=RGB size=512x512 at 0x7F0C2196B970>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC130>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC7C0>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113ECE80>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC250>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC640>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113ECD60>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC670>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113ECE20>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC700>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113EC160>,
 <PIL.Image.Image image mode=RGB size=512x512 at 0x7F0B113ECF10>]

In [ ]:
from sd_server import DiffusersClient

client = DiffusersClient(url)

responses = client(prompt='a photo of an astronaut riding a horse on mars', num_images_per_prompt=4) # this should return a list of images